In [1]:
#Import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from PIL import Image

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
#this works as long as it is inside code/
train_path = os.getcwd()[:-4] + "archive/train/"
test_path = os.getcwd()[:-4] + "archive/test/"

classes = os.listdir(train_path)

images = []
labels = []

for class_ in classes:
    full_path = train_path + str(class_) + "/"
    for file in os.listdir(full_path):
        img = Image.open(full_path + file)

        #needs to resize
        img = img.resize((30, 30))

        #Convert image to grayscale
        img = img.convert('L')
        
        #Convert image to numpy array (Feature Extraction)
        img = np.array(img).flatten()
        img.reshape(img.shape[0], 1)

        #Append image to list
        images.append(img)

        #Append label to list
        labels.append(np.uint8(class_))
        


X = np.array(images)
print("X:", type(X[0][0]))
y = np.array(labels)
y = y.reshape(y.shape[0], 1)
print("y:", type(y[0][0]))

print(X.shape, y.shape)


X: <class 'numpy.uint8'>
y: <class 'numpy.uint8'>
(39209, 900) (39209, 1)


In [3]:
#Split data without sklearn
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    if test_size >= 1 or test_size <= 0:
        raise ValueError("test_size must be smaller than 1 and larger than 0")
    
    
    shuffled_indices = np.random.permutation(len(X))
    test_set_size = int(len(X) * test_size)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return np.array(X[train_indices]), np.array(X[test_indices]), np.array(y[train_indices]), np.array(y[test_indices])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_test = y_test.reshape(y_test.shape[0], 1)

test_images = []
test_labels = []

for file in os.listdir(test_path):
    #Ignore csv file
    if file.endswith(".csv"):
        continue
    
    img = Image.open(test_path + file)

    #needs to resize
    img = img.resize((30, 30))

    #Convert image to grayscale
    img = img.convert('L')
    
    #Convert image to numpy array (Feature Extraction)
    img = np.array(img).flatten()
    img.reshape(img.shape[0], 1)

    #Append image to list
    test_images.append(img)

    #Append label to list
    test_labels.append(np.uint8(class_))

X_original_test, y_original_test = np.array(test_images), np.array(test_labels)

In [5]:
# Normalize features
def featureNormalization(X):
    """
    Take in numpy array of X values and return normalize X values,
    the mean and standard deviation of each feature
    """
    mean= np.mean(X, axis=0)
    std= np.std(X, axis=0)
    
    X_norm = (X - mean) / std
    
    return X_norm , mean , std

In [6]:
# Normalize X_train and X_test (feature scaling)
X_train, mean, std = featureNormalization(X_train)
X_test = (X_test - mean) / std

In [7]:
lg = LogisticRegression(solver='lbfgs', random_state=42, max_iter=1000)
lg.fit(X_train, y_train)

/home/diogo/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/diogo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [8]:
#Predict using train set
pred = lg.predict(X_train)
scores = lg.score(X_train, y_train)
print("Train set Accuracy:", scores)

Train set Accuracy: 0.9897666411629686


In [9]:
#Predict using test set
pred=lg.predict(X_test)
scores = lg.score(X_test, y_test)
print("Test set Accuracy:", scores)


Test set Accuracy: 0.9201632444841219


In [10]:
#Predict using original test set
pred=lg.predict(X_original_test)
scores = lg.score(X_original_test, y_original_test)
print("Test set Accuracy:", scores)


Test set Accuracy: 0.035629453681710214
